##### Copyright 2025 Google LLC.


多智能体系统与工作流模式

创建专门的智能体来协作解决复杂问题。这被称为多智能体系统。

学习在[智能体开发工具包(ADK)](https://google.github.io/adk-docs/)中何时使用多智能体系统
使用LLM作为"管理器"构建你的第一个系统
学习三种核心工作流模式（顺序、并行和循环）来协调你的智能体团队

导入ADK组件包

In [41]:
import os
import dotenv
import warnings
dotenv.load_dotenv()


from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


单个智能体可以做很多事情。但是当任务变得复杂时会发生什么呢？一个试图同时进行研究、写作、编辑和事实核查的单一"单体"智能体会成为一个问题。它的指令提示变得冗长且令人困惑。它难以调试（哪部分失败了？），难以维护，并且经常产生不可靠的结果。

我们可以构建一个多智能体系统，而不是一个"全能"智能体。这是一个由简单、专门的智能体组成的团队，它们像现实世界的团队一样协作。每个智能体都有一个明确的工作（例如，一个智能体只做研究，另一个只写作）。这使得它们更容易构建，更容易测试，并且在一起工作时更强大、更可靠。

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

示例：研究与总结系统

让我们构建一个包含两个专门智能体的系统：

1. 研究智能体 - 使用 Google 搜索信息
2. 总结智能体 - 根据研究结果创建简洁的总结

In [58]:
def google_search(query):
    """
    使用 Google Custom Search API
    
    args:
        query: 搜索关键词
    returns:
        list: 包含搜索结果的列表，每个结果是一个字典，包含标题、URL和描述
    """
    import requests
    
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': os.getenv("SEARCH_API_KEY"),
        'cx': os.getenv("SEARCH_ENGINE_ID"),
        'q': query,
        'num': 3  # 返回结果数量
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        results = []
        for item in data.get('items', []):
            results.append({
                'title': item.get('title'),
                'url': item.get('link'),
                'description': item.get('snippet')
            })
        
        return results
    
    except Exception as e:
        print(f"API搜索出错: {e}")
        return []


In [38]:
# 搜索Agent
from google.adk.models.lite_llm import LiteLlm

research_agent = Agent(
    name="ResearchAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [20]:
# 总结Agent
from google.adk.models.lite_llm import LiteLlm

summarizer_agent = Agent(
    name="SummarizerAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


然后我们将这些智能体组合在一个root智能体或协调器下：

In [39]:
# root 协调器：通过将子智能体作为工具调用来编排工作流。
from google.adk.models.lite_llm import LiteLlm

root_agent = Agent(
    name="ResearchCoordinator",
    model=LiteLlm(
        model="deepseek/deepseek-chat"
    ),
    # instruction告诉root智能体如何使用它的工具（即其他智能体）。
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # 使用AgentTool将子智能体包装为可调用工具，以便root智能体使用。
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [43]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "2025最热电影推荐"
)


 ### Created new session: debug_session_id

User > 2025最热电影推荐
ResearchCoordinator > 我来帮您查找2025年最热门的电影推荐。首先，我将使用研究工具来收集相关信息。
ResearchCoordinator > 现在我将使用总结工具来整理这些研究结果，创建一个清晰的电影推荐总结。
ResearchCoordinator > # 2025年最热门电影推荐

根据我的研究，以下是2025年最值得期待的电影推荐：

## 🎬 好莱坞重磅大片
- **《阿凡达3：火与烬》**  
  - **上映时间**：2025年12月19日（美国）  
  - **关键信息**：詹姆斯·卡梅隆执导的系列第三部，预计将再次引发全球观影热潮。

- **《疯狂动物城2》**  
  - **上映时间**：2025年11月26日（美国）  
  - **关键信息**：迪士尼热门动画续集，已获得影评人好评。

- **《魔法坏女巫2》**  
  - **关键信息**：百老汇改编电影续集，已打破影史纪录。

## 🇨🇳 华语电影亮点
- **《哪吒之魔童降世》续作**  
  - **关键信息**：导演饺子暌违6年回归，延续“我命由我不由天”的精神，是2025年华语影坛最受期待的动画续集。

## 🦸 超级英雄与科幻系列
- **漫威与DC电影**  
  - **关键信息**：2025年预计上映3部漫威电影与1部DC电影，具体片单待公布。

- **《创：战神》**  
  - **关键信息**：睽违15年回归的《创》系列独立续集。

## 🐭 迪士尼动画续作
- **《海洋奇缘2》与《冰雪奇缘3》**  
  - **关键信息**：迪士尼希望通过这两部重磅续集，重振其动画电影板块。

## 📌 其他关注影片
- **《大濛》**  
  - **关键信息**：2025年值得关注的电影之一，具体信息待进一步公布。

---

**总结**：2025年电影市场将迎来多部续作与重磅新片，涵盖科幻、动画、超级英雄与华语电影等类型，预计将掀起新一轮全球观影热潮。建议关注官方发布的具体上映日期和更多详细信息。


你刚刚构建了你的第一个多智能体系统！你使用了一个单一的"协调器"智能体来管理工作流，这是一个强大且灵活的模式。

然而，依赖 LLM 的指令来控制顺序有时可能是不可预测的。接下来，我们将探索一种不同的模式，它可以保证逐步执行。

**顺序工作流 - 流水线**

之前的多智能体系统可以工作，但它依赖于详细的指令提示来强制 LLM 按顺序运行步骤。这可能不可靠。复杂的 LLM 可能会决定跳过某个步骤、以错误的顺序运行它们，或者"卡住"，使过程变得不可预测。

解决方案：固定流水线

当你需要任务以保证的、特定的顺序执行时，你可以使用 `SequentialAgent`。这个智能体就像一条流水线，按照你列出的确切顺序运行每个子智能体。一个智能体的输出自动成为下一个智能体的输入，创建一个可预测且可靠的工作流。

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

示例：使用顺序智能体创建博客文章

让我们构建一个包含三个专门智能体的系统：

1. outline_agent - 为给定主题创建博客大纲
2. 写作智能体 - 撰写博客文章
3. 编辑智能体 - 编辑博客文章草稿以提高清晰度和结构

In [44]:
# outline_agent：创建初始博客文章大纲。
outline_agent = Agent(
    name="OutlineAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [45]:
# Writer Agent: 完成博客文章的撰写
writer_agent = Agent(
    name="WriterAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    #  `{blog_outline}` 自动注入前一个智能体的输出
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [46]:
# Editor Agent: 编辑博客文章草稿
editor_agent = Agent(
    name="EditorAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    # editor_agent 接收 writer_agent 的输出
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


然后我们将这些智能体组合在一个顺序智能体下，它会按照列出的顺序运行这些智能体：

In [47]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


让我们运行智能体并给它一个主题来写博客文章：

In [49]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "写一篇关于多智能体系统对软件开发者益处的博客文章"
)


 ### Created new session: debug_session_id

User > 写一篇关于多智能体系统对软件开发者益处的博客文章
OutlineAgent > 好的，这是为您创建的博客文章大纲。

**1. 引人注目的标题**
从单打独斗到“超级团队”：多智能体系统如何重塑软件开发者的工作方式

**2. 引言钩子**
想象一下，你不是一个人在编码。你的身边有一个由专家组成的“虚拟团队”：一个负责架构设计，一个专注代码审查，一个自动编写测试用例，还有一个实时监控线上问题。这不再是科幻场景，而是多智能体系统正在为软件开发者带来的现实。它并非要取代开发者，而是将你从繁琐、重复的任务中解放出来，让你专注于真正需要创造力和战略思维的核心工作。本文将带你探索，这个“智能团队”如何成为你开发效率的倍增器。

**3. 主要章节**

*   **章节一：效率革命：自动化繁琐工作流**
    *   **智能代码生成与补全**：超越基础的代码提示，智能体能理解项目上下文，生成符合规范的函数、模块甚至API接口代码，极大减少重复性键入和基础逻辑构建时间。
    *   **自动化测试与调试**：智能体可以自动根据代码变更生成测试用例，运行测试并定位失败原因。它们还能模拟用户行为进行探索性测试，并协助分析日志，快速定位生产环境中的疑难Bug。

*   **章节二：质量守护：构建全方位的代码卫士**
    *   **持续的架构与代码审查**：智能体可以扮演“资深架构师”和“审查员”的角色，实时分析代码提交，检查是否符合设计模式、性能最佳实践和安全规范，在问题引入早期就发出预警。
    *   **智能技术债务管理**：智能体能够持续扫描代码库，识别重复代码、复杂函数和过时的依赖库，量化技术债务，并优先提供重构建议，帮助团队主动管理代码健康度。

*   **章节三：认知增强：从执行者到决策者与设计者**
    *   **需求分析与设计辅助**：智能体可以解析模糊的自然语言需求，将其转化为初步的用户故事、技术规格或系统设计图，并与开发者进行问答式交互，澄清细节，加速设计阶段。
    *   **个性化学习与知识管理**：智能体可以成为你的专属技术顾问，根据你当前的项目和技术栈，推荐最相关的文档、案例和解决方案。它还能自动总结项目会议纪要和文档，构

你现在使用顺序智能体创建了一个可靠的"流水线"，其中每个步骤都以可预测的顺序运行。

这对于相互依赖的任务来说非常完美，但如果任务是独立的，它会很慢。 接下来，我们将看看如何同时运行多个智能体来加快你的工作流。

**并行工作流 - 独立研究者**

之前的顺序智能体很好，但它是一条流水线。每个步骤必须等待前一个步骤完成。如果你有几个不相互依赖的任务怎么办？例如，研究三个不同的主题。按顺序运行它们会很慢且效率低下，造成瓶颈，每个任务都在不必要地等待。

当你有独立的任务时，你可以使用 `ParallelAgent` 同时运行所有任务。这个智能体并发执行其所有子智能体，大大加快了工作流。一旦所有并行任务完成，你可以将它们的组合结果传递给最终的"聚合器"步骤。

在以下情况下使用并行模式：任务是独立的，速度很重要，并且可以并发执行。

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

示例：并行多主题研究

让我们构建一个包含四个智能体的系统：

1. 技术研究者 - 研究 AI/ML 新闻和趋势
2. 健康研究者 - 研究最近的医学新闻和趋势
3. 金融研究者 - 研究金融和金融科技新闻和趋势
4. 聚合器智能体 - 将所有研究结果组合成一个总结

In [61]:
# Tech Researcher: 聚焦AI/ML趋势.
tech_researcher = Agent(
    name="TechResearcher",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [62]:
# Health Researcher: 专注于医学突破.
health_researcher = Agent(
    name="HealthResearcher",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [63]:
# Finance Researcher: 专注于金融科技趋势.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [64]:
# aggregator_agent：将所有研究结果组合成一个总结
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


然后我们将这些智能体组合在一个并行智能体下，而并行智能体本身嵌套在一个顺序智能体中。

这种设计确保研究智能体首先并行运行，然后一旦所有研究完成，聚合器智能体将所有研究结果整合到一个报告中：

In [65]:
# ParallelAgent 同时运行其所有子智能体。
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# SequentialAgent 定义了高层次的工作流程：首先运行并行团队，然后运行聚合器。
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


让我们运行智能体并给它一个提示来研究给定的主题：

In [66]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "运行关于技术、健康和金融的每日执行简报"
)


 ### Created new session: debug_session_id

User > 运行关于技术、健康和金融的每日执行简报
FinanceResearcher > 我将为您搜索关于技术、健康和金融的最新信息，提供每日执行简报。
TechResearcher > 我将为您搜索关于技术、健康和金融的最新动态，为您准备每日执行简报。
HealthResearcher > 我将为您搜索最新的技术、健康和金融领域的执行简报信息。
TechResearcher > 让我分别搜索这三个领域的最新信息：
FinanceResearcher > 让我分别搜索这三个领域的最新信息：
HealthResearcher > 让我搜索更具体的执行简报信息：
HealthResearcher > 让我尝试搜索更具体的行业简报：
HealthResearcher > 让我搜索具体的行业新闻来源：
TechResearcher > ## 每日执行简报

**技术领域：**
- 生成式AI成为产业创新核心引擎（德勤2024技术趋势）
- 英伟达股价飙升，AI硬件和云计算驱动增长
- 技术投资聚焦人工智能、自动驾驶等前沿领域

**健康医疗：**
- 主动健康管理成为亚太地区新趋势（贝恩报告）
- Z世代推动个性化医疗需求增长
- 医疗科技、CRO/CDMO领域受投资者关注

**金融经济：**
- 全球并购活动在TMT、金融服务等行业保持活跃
- 中国银行预测2025年全球经济逐步复苏
- 私募资本在医疗健康、工业制造等领域积极布局

**关键洞察：** 人工智能技术正深度融入各行业，推动医疗健康向预防性管理转型，金融投资更加关注科技创新和可持续发展领域。
FinanceResearcher > # 每日执行简报 - 技术、健康、金融

## 技术领域
**人工智能主导转型**：2024年技术趋势以AI为核心，英伟达市值飙升超2万亿美元，反映AI和高性能计算的爆炸性发展。德勤报告显示AI正成为数字经济重要引擎，传统企业需加速数字化转型。

## 健康产业
**数字健康快速发展**：2024年全民数字健康大会在京举行，近500位代表探讨数字健康最新动态。医疗健康产业正经历数字化变革，数字医疗产教融合为高质量发展提供新动力。

## 金融市场
**金融科技投资复苏**：尽管面临地缘政

太好了！你已经看到并行智能体如何通过并发运行独立任务来大幅加快工作流。

到目前为止，我们所有的工作流都是从开始运行到结束然后停止。但是如果你需要多次审查和改进输出怎么办？ 接下来，我们将构建一个可以循环并改进自己工作的工作流。

**循环工作流-改进循环**

到目前为止，我们看到的所有工作流都是从开始运行到结束。`SequentialAgent` 和 `ParallelAgent` 产生最终输出然后停止。这种"一次性"方法不适合需要改进和质量控制的任务。如果我们故事的第一稿很差怎么办？我们没有办法审查它并要求重写。

当任务需要通过反馈和修订循环来改进时，你可以使用 `LoopAgent`。`LoopAgent` 重复运行一组子智能体，直到满足特定条件或达到最大迭代次数。 这创建了一个改进循环，允许智能体系统一次又一次地改进自己的工作。

在以下情况下使用循环模式：需要迭代改进，质量改进很重要，或者需要重复循环。

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

示例：迭代故事改进

让我们构建一个包含两个智能体的系统：

1. 写作智能体 - 撰写短篇故事的草稿
2. 评论智能体 - 审查和评论短篇故事以提出改进建议

In [67]:
# 这个智能体只运行一次，在开始时创建第一个草稿。
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [68]:
# 评论智能体
critic_agent = Agent(
    name="CriticAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


现在，我们需要一种方法让循环根据评论者的反馈实际停止。`LoopAgent` 本身不会自动知道 "APPROVED" 意味着"停止"。

我们需要一个智能体给它一个明确的信号来终止循环。

我们通过两部分来实现：

1. 一个简单的 Python 函数，`LoopAgent` 将其理解为"退出"信号。
2. 一个可以在满足正确条件时调用该函数的智能体。

首先，你将定义 `exit_loop` 函数：

In [69]:
# 这是 RefinerAgent 将调用来退出循环的函数。
def exit_loop():
    """调用该函数时，表示故事已完成，无需进一步修改。"""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


为了让智能体能够调用这个 Python 函数，我们将其包装在 `FunctionTool` 中。然后，我们创建一个拥有这个工具的 `RefinerAgent`。

注意它的指令：这个智能体是循环的"大脑"。它读取来自 `CriticAgent` 的 `{critique}`，并决定是（1）调用 `exit_loop` 工具还是（2）重写故事。

In [70]:
# 该智能体根据评论重写故事或调用 `exit_loop` 函数。
refiner_agent = Agent(
    name="RefinerAgent",
    model=LiteLlm(
        model="deepseek/deepseek-chat",
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


然后我们将这些智能体组合在一个循环智能体下，而循环智能体本身嵌套在一个顺序智能体中。

这种设计确保系统首先产生初始故事草稿，然后改进循环运行到指定的 `max_iterations` 次数：

In [71]:
# LoopAgent 包含将重复运行的智能体：Critic -> Refiner。
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # 防止无限循环
)

# root_agent是一个顺序智能体，定义了总体工作流程：Initial Write -> Refinement Loop。
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


让我们运行智能体并给它一个主题来写短篇故事：

In [72]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "写一个关于灯塔看守人发现神秘发光地图的短篇故事"
)


 ### Created new session: debug_session_id

User > 写一个关于灯塔看守人发现神秘发光地图的短篇故事
InitialWriterAgent > 老灯塔看守人埃里克在风暴夜爬上旋转楼梯时，木阶意外塌陷了一块。他伸手摸索，指尖触到的不是碎木，而是一卷用油布包裹的硬物。回到小屋的煤油灯下展开，羊皮纸上浮现的不是海岸线，而是用某种发光磷粉绘制的、随心跳般明暗起伏的星图。最亮的那颗星，正指向灯塔下方礁石群中，那个传说淹死过无数水手的漩涡。此刻，漩涡方向的海面上，无端亮起了一盏与他手中星图同步闪烁的幽绿光点。
CriticAgent > APPROVED
RefinerAgent > 故事已获批准，无需进一步修改。
CriticAgent > APPROVED
RefinerAgent > 故事已获批准，无需进一步修改。


你现在实现了一个循环智能体，创建了一个可以迭代审查和改进自己输出的复杂系统。这是确保高质量结果的关键模式。

你现在拥有完整的工作流模式工具包。让我们把它们整合起来，回顾如何为你的用例选择合适的模式。

**总结-选择合适的模式**

决策树：哪种工作流模式？

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### 快速参考表

| 模式 | 何时使用 | 示例 | 关键特性 |
|---------|-------------|---------|-------------|
| **基于 LLM (sub_agents)** | 需要动态编排 | 研究 + 总结 | LLM 决定调用什么 |
| **顺序** | 顺序重要，线性流水线 | 大纲 → 写作 → 编辑 | 确定性顺序 |
| **并行** | 独立任务，速度重要 | 多主题研究 | 并发执行 |
| **循环** | 需要迭代改进 | 写作 + 评论改进 | 重复循环 |

在这个笔记本中，你从单个智能体跨越到了多智能体系统

你看到了**为什么**一个专家团队比一个"全能"智能体更容易构建和调试。最重要的是，你学会了如何成为那个团队的**导演**。

你使用了 `SequentialAgent`、`ParallelAgent` 和 `LoopAgent` 来创建确定性工作流，甚至使用 LLM 作为"管理器"来做出动态决策。你还通过使用 `output_key` 在智能体之间传递状态并使它们协作，掌握了"管道"技术。